In [1]:
import sys
sys.path.append('..')

In [2]:
from DisplayPane import DisplayPane
from VisionSystem.DataSet import DataSet

dset = DataSet('../data/calibration2')
frame, _ = next(iter(dset))
frame.shape

(1232, 1281, 3)

In [3]:
import re

all_labels = []
rel_coords2pix_coords = {}

for frames in dset.labels.values():
    for frame in frames:
        for type_name, (_, points) in frame.labels.items():
            all_labels += points
            for point in points:
                match = re.search(r'\((\d+), (\d+)\)', point.coords_str())
                pix_coords = int(match.group(1)), int(match.group(2))
                
                match2 = re.search(r'(\-?\d+),(\-?\d+)', point.tags['rel_coords'])
                rel_coords = float(match2.group(1)), float(match2.group(2))
                
                if rel_coords in rel_coords2pix_coords:
                    rel_coords2pix_coords[rel_coords].append(pix_coords)
                else:
                    rel_coords2pix_coords[rel_coords] = [pix_coords]

In [4]:
from VisionSystem.Label import FrameLabels, Point

dset.labels[dset.filepath] = [FrameLabels({dset.filepath: (Point, all_labels)})]

In [5]:
# display all the raw data to visually identify outliers
# DisplayPane(dataset=dset)

In [6]:
from statistics import mean

# hey - that data is relatively clean! lets average the points that are the same to get rid of overlapping points
for rel_coords, points in rel_coords2pix_coords.items():
    sum_x, sum_y = 0, 0
    for point in points:
        x, y = point
        sum_x += x
        sum_y += y
    rel_coords2pix_coords[rel_coords] = int(sum_x / len(points)), int(sum_y / len(points))

In [7]:

dset.labels[dset.filepath] = [
    FrameLabels({
        dset.filepath: (
            Point,
            [Point(pix_coords) for pix_coords in rel_coords2pix_coords.values()]
        )
    })
]

# visually confirm that all points at the same coordinates have been merged
DisplayPane(dataset=dset)

DisplayPane(children=(HBox(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'r…

In [8]:
from cv2 import calibrateCamera, getOptimalNewCameraMatrix, initUndistortRectifyMap, CV_32FC1 as undistort_map_backend
import numpy as np

bgr, labels = next(iter(dset))
y, x, z = bgr.shape
resolution = x, y

object_points = np.array([[(co[0] * 0.1, co[1] * 0.1) + (0,) for co in rel_coords2pix_coords.keys()]])
image_points = np.array([list(rel_coords2pix_coords.values())])

ret, cam_mtx, dist_coeffs, _rvecs, _tvecs = calibrateCamera(
    objectPoints=object_points.astype(np.float32),
    imagePoints=image_points.astype(np.float32),
    imageSize=resolution,
    cameraMatrix=None,
    distCoeffs=None
)

new_cam_mtx, _roi = getOptimalNewCameraMatrix(cam_mtx, dist_coeffs, resolution, 1, resolution)

# # use openCV to use cell-based interpolation homography
# undist_map, rectif_map = initUndistortRectifyMap(
#     cam_mtx, dist_coeffs, None, new_cam_mtx,
#     resolution, undistort_map_backend
# )

In [9]:
DisplayPane(img=bgr)

DisplayPane(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, lay…

In [10]:
cam_mtx, new_cam_mtx, dist_coeffs

(array([[2.07944316e+03, 0.00000000e+00, 6.48433013e+02],
        [0.00000000e+00, 2.08467081e+03, 6.39799003e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[510.30801392,   0.        , 404.89650131],
        [  0.        , 722.31158447, 573.15101777],
        [  0.        ,   0.        ,   1.        ]]),
 array([[-5.01158318e+00,  1.64444118e+01,  7.68400301e-04,
         -1.93648970e-03, -2.01211004e+01]]))

In [11]:
import cv2
bgr, labels = next(iter(dset))

DisplayPane(img=cv2.undistort(bgr, cam_mtx, dist_coeffs, None, new_cam_mtx))

DisplayPane(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, lay…

In [12]:
import cv2
import ipywidgets as ipy
from IPython.display import display
from VisionSystem import ColorSpaces

pane = DisplayPane(img=bgr)
display(pane)

kwargs = {
    # camera matrix
    'cam_fx': 2.07944325e+03,
    'cam_fy': 2.08467090e+03,
    'cam_cx': 6.48433013e+02,
    'cam_cy': 6.39799003e+02,
    
    # distortion coeffs
    'dist_k1': -5.01158356e+00,
    'dist_k2': 1.64444140e+01,
    'dist_p1': 7.68400444e-04,
    'dist_p2': -1.93648972e-03,
    'dist_k3': -2.01211042e+01
}

cam_fx = cam_mtx[0, 0]
cam_fy = cam_mtx[1, 1]
cam_cx = cam_mtx[0, 2]
cam_cy = cam_mtx[1, 2]

[[dist_k1], [dist_k2], [dist_p1], [dist_p2], [dist_k3]] = dist_coeffs.T

# print('dist_k1', dist_k1)
# print('dist_k2', dist_k2)
# print('dist_p1', dist_p1)
# print('dist_p2', dist_p2)
# print('dist_k3', dist_k3)

def undistort(cam_fx=cam_fx, cam_fy=cam_fy, alpha=1.0, dist_p1=dist_p1, dist_p2=dist_p2,
              dist_k1=dist_k1, dist_k2=dist_k2, dist_k3=dist_k3, dist_k4=1, dist_k5=1, dist_k6=1):
    cx, cy = rel_coords2pix_coords[(0, 0)]
    cam_mtx = np.array([
        [cam_fx, 0, cx],
        [0, cam_fy, cy],
        [0, 0, 1]
    ])
    
    dist_coeffs2 = np.array([dist_k1, dist_k2, dist_p1, dist_p2, dist_k3, dist_k4, dist_k5, dist_k6]).reshape((1, 8))
    
    new_cam_mtx, _roi = getOptimalNewCameraMatrix(cam_mtx, dist_coeffs2[:, :5], resolution, alpha, resolution)
    
    undistorted = cv2.undistort(bgr, cam_mtx, dist_coeffs2, None, new_cam_mtx)
    
    pane.raw_frame.link(undistorted, ColorSpaces.BGR)
    pane.update_data_and_display()


# print('dist_k2', dist_k2)
# print('dist_p1', dist_p1)
# print('dist_p2', dist_p2)
# print('dist_k3', dist_k3)
ipy.interact(undistort,
             cam_fx=ipy.FloatSlider(value=cam_fx, min=0.5*cam_fx, max=2*cam_fx, step=1e-10, readout_format=".6f"),
             cam_fy=ipy.FloatSlider(value=cam_fy, min=0.5*cam_fy, max=2 * cam_fy, step=1e-10, readout_format=".6f"),
             alpha=ipy.FloatSlider(value=2, min=0, max=100, step=1e-10, readout_format=".10f"),
             dist_p1=ipy.FloatSlider(value=0, min=-0.2, max=0.2, step=1e-10, readout_format=".10f"),
             dist_p2=ipy.FloatSlider(min=-0.2, max=0.2, step=1e-10, readout_format=".10f"),
             dist_k1=ipy.FloatSlider(value=0, min=-1, max=5, step=1e-10, readout_format=".10f"),
             dist_k4=ipy.FloatSlider(min=-10, max=10, step=1e-10, readout_format=".10f"),
             dist_k2=ipy.FloatSlider(value=0, min=-30, max=30, step=1e-10, readout_format=".10f"),
             dist_k5=ipy.FloatSlider(min=-40, max=40, step=1e-10, readout_format=".10f"),
             dist_k3=ipy.FloatSlider(value=0, min=2*min(-dist_k3, dist_k3), max=2*max(-dist_k3, dist_k3), step=1e-10, readout_format=".10f"),
             dist_k6=ipy.FloatSlider(min=-20, max=20, step=1e-10, readout_format=".10f"))

DisplayPane(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, lay…

interactive(children=(FloatSlider(value=2079.443158004651, description='cam_fx', max=4158.886316009302, min=10…

<function __main__.undistort(cam_fx=2079.443158004651, cam_fy=2084.670809072715, alpha=1.0, dist_p1=0.0007684003008221206, dist_p2=-0.001936489704360318, dist_k1=-5.0115831768331995, dist_k2=16.444411801975207, dist_k3=-20.12110039346355, dist_k4=1, dist_k5=1, dist_k6=1)>

In [13]:
rel_coords2pix_coords[(0, 0)]

(637, 639)

In [14]:
# notable vals
cam_fx = 2079.443158
cam_fy = 2084.670809

dist_k2 = 18.4562400000

In [15]:
cx, cy = rel_coords2pix_coords[(0, 0)]

real_dist2img_dist = {}

for (dx, dy), (x, y) in rel_coords2pix_coords.items():
    real_dist2img_dist[
        np.sqrt(pow(dx, 2) + pow(dy, 2))
    ] = (
        np.sqrt(pow(x - cx, 2) + pow(y - cy, 2))
    )


In [16]:
from bqplot import pyplot as plt

fig = plt.figure('real_dists / img_distss')
real_dists, img_dists = zip(*list(sorted(real_dist2img_dist.items(), key=lambda item: item[1])))
real_dists, img_dists = np.array(real_dists), np.array(img_dists)
patch = plt.plot(img_dists, real_dists)
fig

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top':…

In [17]:
from scipy.optimize import curve_fit

def fit_model(x, a, b, c):
    return a * np.exp(b*x) + c

RESCALE = 1e2

model, cov = curve_fit(fit_model, [dist / RESCALE for dist in img_dists], real_dists)

predictions = np.array([fit_model(img_dist / RESCALE, *model) for img_dist in img_dists])
patch.y = np.append(patch.y.reshape((1,len(patch.y))), predictions.reshape((1,len(patch.y))), axis=0)

In [18]:
with open('data.csv', 'w') as f:
    for real_dist, img_dist, pred in zip(real_dists, img_dists, predictions):
        f.write(f'{img_dist},{real_dist},{pred}\n')

In [19]:
fit_model(0, *model)

0.9344245742327053

In [20]:
import cv2

img = cv2.imread('../data/calibration/2.png')
crop = (0.13, 0), (.9, 1)
height, width, _ = img.shape
img = img[:, int(0.13 * width):int(.9 * width)]
print(img.shape)
DisplayPane(img=img)

(1232, 1281, 3)


DisplayPane(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, lay…

In [21]:
from numpy import *

def find_radial_bounding_box(contours, theta, roi_offset):
    rx, ry = roi_offset
    
    min_dist_1, min_dist_2 = 99999999, 999999999
    max_dist_1, max_dist_2 = -99999999, -999999999

    for contour in contours:
        for [[x, y]] in contour:
            # relative to camera point (0, 0)
            x += rx - cx
            y += ry - cy
            
            # okay so how this algorithm works is best explained by
            # a geometry picture which i will put in the report
            a = sqrt(pow(x, 2) + pow(y, 2))
            direc = arctan2(y, x)
            phi = arctan2(y, x) - theta
            
            dist_1 = a * cos(phi)
            dist_2 = a * tan(phi)
            
            if dist_1 > max_dist_1:
                max_dist_1 = dist_1
            elif dist_1 < min_dist_1:
                min_dist_1 = dist_1
            
            if dist_2 > max_dist_2:
                max_dist_2 = dist_2
            elif dist_2 < min_dist_2:
                min_dist_2 = dist_2
    
    # G E O M E T R Y
    dirx, diry = cos(theta), sin(theta)
    rect_inner_midpoint = (min_dist_1 * dirx), (min_dist_1 * diry)
    mx, my = rect_inner_midpoint
    
    d90 = pi / 2
    
    left_x, right_x = abs(min_dist_2) * cos(theta + d90), max_dist_2 * cos(theta - d90)
    left_y, right_y = abs(min_dist_2) * sin(theta + d90), max_dist_2 * sin(theta - d90)
    
    bottom_left_corner = (mx + left_x), (my + left_y)
    bottom_right_corner = (mx + right_x), (my + right_y)
    
    rect_outer_midpoint = (max_dist_1 * dirx), (max_dist_1 * diry)
    mx, my = rect_outer_midpoint
    
    top_left_corner = (mx + left_x), (my + left_y)
    top_right_corner = (mx + right_x), (my + right_y)
    
    # uhhuh
    return min_dist_1, (top_left_corner, top_right_corner, bottom_right_corner, bottom_left_corner)

In [22]:
def convert_img_dist_to_real_dist(img_dist):
    return fit_model(img_dist / RESCALE, *model) / 10

In [23]:
from VisionSystem import VisionSystem, VisualObject
from VisionSystem.DetectionModel import ThreshBlob, DetectionResult

ball_model = ThreshBlob.load('../models/ball.threshblob.pkl')

def apply(frame):
    global mask
    self = ball_model
    mask = self.thresholder.apply(frame)
    height, width = mask.shape
    params = cv2.SimpleBlobDetector_Params()
    for name, val in self.blob_detector_params.items():
        setattr(params, name, val)
    blob_detector = cv2.SimpleBlobDetector_create(params)

    results = []
    for keypoint in blob_detector.detect(mask):
        x, y = keypoint.pt
        x, y = int(x), int(y)

        # detect contours to find the true bounding rect
        contour_padding = int(keypoint.size * 0.75)
        (roi_x1, roi_y1), (roi_x2, roi_y2) = (
            (max(x - contour_padding, 0), max(y - contour_padding, 0)),
            (min(x + contour_padding, width), min(y + contour_padding, height))
        )
        
        _, contours, _ = cv2.findContours(
            mask[roi_y1:roi_y2, roi_x1:roi_x2].copy(),
            cv2.RETR_TREE,
            cv2.CHAIN_APPROX_SIMPLE
        )
        theta = arctan2(y - cy, x - cx)
        img_dist, ((x1, y1), (x2, y2), (x3, y3), (x4, y4)) =\
            find_radial_bounding_box(contours, theta, (roi_x1, roi_y1))
        
        coords = \
            (int(x1 + cx), int(y1 + cy)),\
            (int(x2 + cx), int(y2 + cy)),\
            (int(x3 + cx), int(y3 + cy)),\
            (int(x4 + cx), int(y4 + cy))
        
        result = DetectionResult(
            coords=coords,
            bitmask=mask
        )
        result.bearing = -theta # ??? But it works!!!! DOn't TOUCH IT!!!
        result.distance = convert_img_dist_to_real_dist(img_dist)
        results.append(result)

    return results

# monkey-patch experiment with theshblob
ball_model.apply = apply

height, width, _ = bgr.shape

ball = VisualObject(
    real_size=(0.043, 0.043, 0.043),
    detection_model=ball_model,
    result_limit=1
)
vision = VisionSystem(objects_to_track={'ball': ball}, resolution=(width, height))

In [24]:
from DisplayPane.Interactor.VisionSystemTuner import VisionSystemTuner

DisplayPane(vision_system=vision, img=img, interactors=[VisionSystemTuner(vision)])

DisplayPane(children=(HBox(children=(VBox(children=(Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'r…

ValueError: unsupported dtype: object

ValueError: unsupported dtype: object